# Purchase data analysis 1: Data

The aim of this notebook is to check the quality of data. 

![](../sql/data-model.png)

In [1]:
from typing import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from IPython.display import display, HTML
plt.style.use("fivethirtyeight")

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

#pd.set_option('display.max_rows', None)

In [3]:
import sys
sys.path.append("..")
from lib.database import Database
from lib.processing import Inspector

## Orders

This table describes all orders. Each row corresponds to one purchase. 

In [4]:
with Database("../sql/database.sqlite") as db:
    df_orders = db.read_table("Orders", is_datetime=lambda c: c.lower().endswith("date"))
    df_zip = db.read_table("ZipCounty")

In [5]:
df_orders.head()

,orderId,customerId,campaignId,orderDate,city,state,zipCode,paymentType,totalPrice,numOrderlines,numUnits
0,999992,107237,2173,2010-01-28,WILMINGTON,DE,19806,DB,15.00,1,1
1,999993,20350,2173,2010-01-28,SWARTHMORE,PA,19081,DB,19.12,1,1
2,999994,109529,2173,2010-01-28,SAN JOSE,CA,95118,VI,7.95,1,1
3,999995,109159,2173,2010-01-29,INDIANAPOLIS,IN,46214,VI,19.12,1,1
4,999996,113757,2173,2010-01-29,IOWA CITY,IA,52245,MC,19.12,1,1


In [6]:
df_orders.shape ## (number of rows, number of columns)

(192983, 11)

In [7]:
inspector = Inspector(df_orders)
inspector.set_variable_type("customerId","categorical")
inspector.set_variable_type("campaignId","categorical")
inspector

,dtype,count_na,rate_na,n_unique,distinct,variable,sample_value
orderId,int64,0,0.000000,192983,True,continuous,1140291
customerId,int64,0,0.000000,189560,False,categorical,176657
campaignId,int64,0,0.000000,239,False,categorical,2001
orderDate,datetime64[ns],0,0.000000,2541,False,continuous,2011-03-14 00:00:00
city,object,19,0.000098,12822,False,categorical,HUNINGTON
state,object,1119,0.005798,91,False,categorical,HI
zipCode,object,146,0.000757,15593,False,categorical,49007
paymentType,object,0,0.000000,6,False,categorical,??
totalPrice,float64,0,0.000000,7653,False,continuous,364.5
numOrderlines,int64,0,0.000000,41,False,continuous,28


The number of unique values of `orderDate` is relatively small. This is because the column represents really dates rather than time stamps.

In [8]:
df_orders["orderDate"].apply(lambda x: x.time()).unique() ## unique values of HH:MM:SS 

array([datetime.time(0, 0)], dtype=object)

Regarding relation among missing values in `city`, `state` and `zipCode`. There is no obvious relation between missing values of these variables: It is not the case that a missing value of one of them implies any missing value of other columns.

In [9]:
geo_cols = ["city","state","zipCode"]
pd.isna(df_orders[geo_cols]).groupby(geo_cols).apply(lambda x:pd.Series(x.shape[0], index=["count"]))

count
city  state zipCode        
False False False    191846
            True          2
      True  False       974
            True        142
True  False False        15
            True          1
      True  False         2
            True          1

But the real problem of these columns is that we can not trust their values. For example, the 100 example values of `zipCode` with a non-digit letter are following.

In [10]:
import re
pd.Series([v for v in df_orders["zipCode"] if not re.search(r"^\d{1,}$",str(v))]).unique()[:100]

array(['T2N3C', 'N6G2S', '193-0', '458-0', '167-0', 'M5J2H', 'T8C1G',
       'V6H', 'K1N7D', 'V6J2G', '675-0', 'M5T2R', 'M5R1S', 'M5N1S',
       'L9B2G', 'V5B2T', 'J4X2A', 'M4G3R', 'S7K 7', 'ENGLA', 'M5R3P',
       'L7M4A', 'M4V2R', 'M5N1W', 'BD111', 'BAHAM', 'm2n 1', 'R2V3M',
       'T2L2K', None, 'M5P3C', 'V6N2T', 'V6K2S', 'H2Y2L', 'M4S1G',
       'M5R3R', 'M4W1P', 'M4L3V', 'T2R1L', 'p3e2p', 'T2P3R', 'V0R2P',
       'N2L5X', 'M5R1T', 'L4T1G', 'M5A4E', 'H4P 1', 'L9Y4T', 'M2N1N',
       'E2L4M', 'M4W1N', 'V6B4M', 'L8P2G', 'K7M5Y', 'M5S3E', 'A1A3H',
       'M4J3E', 'M2L2X', 'M5A4L', 'M6A2J', 'JAPAN', 'M1E1K', 'H4N1J',
       'T6H5J', 'V6S1E', 'K1Y3X', 'M6R1C', 'NSW 2', 'T2P0P', 'M5R2G',
       'R3H0M', 'LS82S', 'M5E1T', 'K2G6B', 'L1S4X', 'N5A4J', 'M8V2Z',
       'M4T1G', 'M5N2K', 'V3N4P', 'NE391', 'M4K2Y', 'N6A2S', 'M4W2P',
       'M5R1Z', 'N169L', 'K1N6M', 'SW156', 'G1T1C', 'V6A1B', 'L3R1Z',
       'GERMA', 'M8X1R', 'V5K1S', 'B4E1W', 'L1V6P', 'V6M4B', 'K9H3L',
       'J0X2G', 'CB124'],

Moreover we can find values which are likely to be country names.

In [11]:
pd.Series([v for v in df_orders["zipCode"] if re.search(r"^[a-zA-Z]{1,}$",str(v))]).unique()

array(['ENGLA', 'BAHAM', None, 'JAPAN', 'GERMA', 'BERMU', 'SWITZ',
       'BELGI', 'AUSTR', 'NETHE', 'ITALY', 'MEXIC', 'SPAIN', 'SOAME',
       'MIMIH', 'NOVAS', 'UARAB', 'CA', 'ny', 'Germa', 'SWIAZ', 'LEIXL',
       'SWEDE', 'XYZ', 'ITALI', 'WELLI', 'HONGK', 'DENMA', 'NY', 'irela',
       'TX', 'GRENA', 'none', 'nil', 'DUBLI', 'ma', 'Finla', 'Irela',
       'XXXXX'], dtype=object)

A similar problem occurs also for `state`. This column contains values which are not in US.

In [12]:
df_orders["state"].unique()

array(['DE', 'PA', 'CA', 'IN', 'IA', 'NY', 'MI', 'GA', 'MA', 'VA', 'NJ',
       'ME', 'CT', 'OH', 'TX', 'MD', 'FL', 'MO', 'IL', 'VT', 'SC', 'DC',
       'KS', 'TN', 'WI', 'WA', 'NV', None, 'PR', 'OK', 'NM', 'MN', 'NE',
       'KY', 'NC', 'AZ', 'HI', 'LA', 'CO', 'RI', 'ND', 'UT', 'AB', 'AK',
       'ON', 'MT', 'AL', 'OR', 'AR', 'BC', 'NH', 'MS', 'ID', 'QC', 'SK',
       'WV', 'BD', 'MB', 'SD', 'PQ', 'AE', 'NB', 'NF', 'AP', 'NS', 'WY',
       'AA', 'US', 'VI', 'QL', 'UK', 'SO', 'SP', 'EN', 'LC', 'PE', 'GU',
       'DF', 'YU', 'SR', 'NT', 'MG', 'FR', 'CN', 'CH', 'VC', 'KM', 'KW',
       '.', 'PC', 'GD', 'NL'], dtype=object)

There are (at least) two possible measures.

1. We do not use these columns, because we can not trust them.
2. We restrict the data set, so that the triad (city, state, zipCode) can be found in the dataset ZipCounty. (Here we trust data set ZipCountry.)

The former is easiest but we lose geographic information. The latter can be reasonable, but many data will be lost. In fact around 1/3 of the data will be lost.

In [13]:
df_geo = df_zip[["poname","state","zipcode"]].drop_duplicates()\
                                             .rename({"poname":"city","zipcode":"zipCode"},axis=1)
df_geo["zipCode"] = df_geo["zipCode"].apply(str)
df_orders_valid = pd.merge(df_orders, df_geo, how="inner")

np.round(100*(1-df_orders_valid.shape[0]/df_orders.shape[0]),2) ## % of rows without geo info in ZipCounty

33.94

## Customers

This table describes all customers. Each row corresponds to a single customer.

In [14]:
with Database("../sql/database.sqlite") as db:
    df_customers = db.read_table("Customers")
    
df_customers.tail()

,customerId,householdId,gender,firstName
189554,189555,68324194,F,PEGGY
189555,189556,59708008,F,ELIZABETH
189556,189557,67518983,F,KIA
189557,189558,55688284,F,MARY
189558,189559,55706354,M,DAVID


In [15]:
inspector_customers = Inspector(df_customers)
inspector_customers

,dtype,count_na,rate_na,n_unique,distinct,variable,sample_value
customerId,int64,0,0.000000,189559,True,continuous,58224
householdId,int64,0,0.000000,156258,False,continuous,20755535
gender,object,16204,0.085483,2,False,binary,F
firstName,object,2915,0.015378,10145,False,categorical,GERALD


The most interesting variable is `householdId`. It is usually difficult to detect who belongs to the same household. But we have to note that a household can be a company or a quite large group.

In [16]:
df_customers["householdId"].value_counts().sort_values(ascending=False)[:10]

19885296    746
49927024    169
36201520     38
22269801     28
19440306     24
36209331     21
75467926     21
19626230     17
20656767     17
19535255     16
Name: householdId, dtype: int64

## Orderlines

This table describes items of all purchases. Roughly speaking, each row corresponds a pair (`orderId`, `productId`), but this is not rigorous.

In [17]:
with Database("../sql/database.sqlite") as db:
    df_orderlines = db.read_table("Orderlines")
    
df_orderlines.tail()

,orderlineId,orderId,productId,shipDate,billDate,unitPrice,numUnits,totalPrice
286012,2017155,1643154,11012,2016-09-20 00:00:00,2016-09-21 00:00:00,16.95,1,16.95
286013,2017156,1643155,13630,2016-09-20 00:00:00,2016-09-21 00:00:00,16.14,1,16.14
286014,2017157,1643150,14028,2016-09-20 00:00:00,2016-09-21 00:00:00,22.95,1,22.95
286015,2017158,1643156,14028,2016-09-20 00:00:00,2016-09-21 00:00:00,20.65,1,20.65
286016,2017159,1643157,12172,2016-09-20 00:00:00,2016-09-21 00:00:00,49.45,1,49.45


All columns have no missing values except `unitPrice`. 

In [18]:
inspector_orderlines = Inspector(df_orderlines)
inspector_orderlines

,dtype,count_na,rate_na,n_unique,distinct,variable,sample_value
orderlineId,int64,0,0.000000,286017,True,continuous,1065362
orderId,int64,0,0.000000,192983,False,continuous,1349125
productId,int64,0,0.000000,4040,False,continuous,10505
shipDate,object,0,0.000000,1887,False,categorical,2011-02-21 00:00:00
billDate,object,0,0.000000,1901,False,categorical,2010-03-23 00:00:00
unitPrice,float64,4984,0.017426,3194,False,continuous,16.65
numUnits,int64,0,0.000000,158,False,continuous,260
totalPrice,float64,0,0.000000,4082,False,continuous,11.66


The following table shows the count of rows of each pair of (`orderId`, `productId`). As you see that the pair is not a primary key.

In [19]:
def count_rows_by(df:pd.DataFrame, by=List[str]) -> pd.DataFrame:
    """
    SELECT by[0], ..., count(*) FROM DF GROUP BY by[0], ...
    
    :return: DataFrame[by[0],...,count]
    """
    df_count = df.groupby(by).apply(lambda dg: dg.shape[0]).rename("count")\
                 .reset_index()\
                 .sort_values(by="count", ascending=False)
    return df_count

pks = ["orderId","productId"]
df_count_pairs = count_rows_by(df_orderlines, pks)
df_count_pairs.iloc[:10,:]

,orderId,productId,count
261337,1502133,12141,40
171158,1155038,12171,12
209489,1253072,12171,11
211904,1268137,12172,11
230200,1349246,12826,9
199069,1200015,12171,8
240086,1411136,12172,8
165926,1149007,12826,8
170525,1153097,12172,8
176149,1166170,12172,7


But theoretically we have to be able to merge these multiple rows. Lets see some examples.

In [20]:
idx = 4
orderId, productId, _ = df_count_pairs.iloc[idx,:]

df_orderlines.query("orderId == @orderId and productId == @productId")

,orderlineId,orderId,productId,shipDate,billDate,unitPrice,numUnits,totalPrice
234070,1541282,1349246,12826,2015-07-26 00:00:00,2015-07-27 00:00:00,107.95,1,107.95
234071,1541283,1349246,12826,2015-07-26 00:00:00,2015-07-27 00:00:00,107.95,1,107.95
234072,1541284,1349246,12826,2015-07-26 00:00:00,2015-07-27 00:00:00,107.95,1,107.95
234073,1541285,1349246,12826,2015-07-26 00:00:00,2015-07-27 00:00:00,107.95,1,107.95
234074,1541286,1349246,12826,2015-07-26 00:00:00,2015-07-27 00:00:00,107.95,1,107.95
234075,1541287,1349246,12826,2015-07-26 00:00:00,2015-07-27 00:00:00,107.95,1,107.95
234076,1541288,1349246,12826,2015-07-26 00:00:00,2015-07-27 00:00:00,107.95,1,107.95
234077,1541289,1349246,12826,2015-07-26 00:00:00,2015-07-27 00:00:00,107.95,1,107.95
234078,1541290,1349246,12826,2015-07-26 00:00:00,2015-07-27 00:00:00,107.95,1,107.95


In [21]:
idx = 10
orderId, productId, _ = df_count_pairs.iloc[idx,:]

df_orderlines.query("orderId == @orderId and productId == @productId").sort_values(by="shipDate")

,orderlineId,orderId,productId,shipDate,billDate,unitPrice,numUnits,totalPrice
204106,1309857,1206372,10643,2014-12-13 00:00:00,2015-01-03 00:00:00,540.0,1,540.0
208047,1337107,1206372,10643,2014-12-13 00:00:00,2015-01-21 00:00:00,540.0,1,540.0
208048,1337108,1206372,10643,2014-12-13 00:00:00,2015-01-21 00:00:00,540.0,1,540.0
208049,1337109,1206372,10643,2014-12-13 00:00:00,2015-01-21 00:00:00,540.0,1,540.0
204107,1309858,1206372,10643,2014-12-19 00:00:00,2015-01-03 00:00:00,540.0,1,540.0
204108,1309859,1206372,10643,2014-12-19 00:00:00,2015-01-03 00:00:00,540.0,1,540.0


Sometimes we have lots of duplicates and sometimes `shipDate` and `billDate` are different.

Next let us look at the missing values of `unitPrice`.

In [22]:
df_orderlines[pd.isna(df_orderlines["unitPrice"])].head(20)

,orderlineId,orderId,productId,shipDate,billDate,unitPrice,numUnits,totalPrice
171461,1197104,1153088,12502,2014-10-29 00:00:00,2014-10-30 00:00:00,NaN,0,0.0
171478,1197121,1153046,12837,2014-10-30 00:00:00,2014-10-30 00:00:00,NaN,0,0.0
171533,1197176,1153115,12851,2014-10-29 00:00:00,2014-10-30 00:00:00,NaN,0,0.0
171552,1197195,1153001,12479,2014-10-29 00:00:00,2014-10-30 00:00:00,NaN,0,0.0
171553,1197196,1153001,12819,2014-10-29 00:00:00,2014-10-30 00:00:00,NaN,0,0.0
171571,1197214,1153104,12495,2014-10-29 00:00:00,2014-10-30 00:00:00,NaN,0,0.0
171589,1197232,1153000,12851,2014-10-29 00:00:00,2014-10-30 00:00:00,NaN,0,0.0
171594,1197237,1153104,12819,2014-10-29 00:00:00,2014-10-30 00:00:00,NaN,0,0.0
171651,1197294,1153060,12832,2014-10-29 00:00:00,2014-10-30 00:00:00,NaN,0,0.0
171657,1197300,1152998,12488,2014-10-29 00:00:00,2014-10-30 00:00:00,NaN,0,0.0


As we see, something wrong happened. Probably the shipping or billing failed, so that the customer received the product quite later. Moreover we can guess that the missing value of `unitPrice` occurs if and only if the `numUnits` is zero. This guess is right.

In [23]:
## list of values of `numUnits` where `unitPrice` is missing.
df_orderlines["numUnits"][pd.isna(df_orderlines["unitPrice"])].unique()

array([0])

In [24]:
## list of values of `unitPrice` where `numUnits` is zero.
df_orderlines["unitPrice"][df_orderlines["numUnits"]==0].unique()

array([nan])

Therefore we can safely remove rows without `unitPrice` unless we want to see something strange.

In [25]:
df_orderlines.dropna(how="any", inplace=True)

According to the above analysis we can say that the table `Orderlines` describes each *shipping* per product. Here "shipping" does not mean `shipDate`. Moreover we have to keep in mind that the `unitPrice` of the same product can vary.

In [26]:
count_rows_by(df_orderlines[["productId","unitPrice"]].drop_duplicates(), by=["productId"]).head()

,productId,count
3593,13629,153
2131,12139,115
2166,12174,87
2806,12826,82
1998,12005,78


In [27]:
df_orderlines.query("productId == '13629'")["unitPrice"].value_counts().head(10)

31.50    1572
35.00    1249
26.25      50
0.00       46
32.65      22
31.49      18
32.50      16
29.75      13
21.00      11
32.40       9
Name: unitPrice, dtype: int64

## Products



In [28]:
with Database("../sql/database.sqlite") as db:
    df_products = db.read_table("Products")
    
df_products.tail()

,productId,productName,productGroupCode,productGroupName,inStockFlag,fullPrice
4035,14036,None,AR,ARTWORK,Y,1500
4036,14037,None,AR,ARTWORK,Y,1200
4037,14038,None,AR,ARTWORK,Y,495
4038,14039,None,BK,BOOK,Y,14
4039,14040,None,AR,ARTWORK,Y,950


In [29]:
inspector_products = Inspector(df_products)
inspector_products

,dtype,count_na,rate_na,n_unique,distinct,variable,sample_value
productId,int64,0,0.000000,4040,True,continuous,13018
productName,object,4040,1.000000,0,False,constant,NaN
productGroupCode,object,0,0.000000,9,False,categorical,BK
productGroupName,object,1,0.000248,8,False,categorical,GAME
inStockFlag,object,0,0.000000,2,False,binary,N
fullPrice,int64,0,0.000000,305,False,continuous,165


The column `productName` is empty. There is one row which `productGroupName` is missing.

In [30]:
df_products[pd.isna(df_products["productGroupName"])]

,productId,productName,productGroupCode,productGroupName,inStockFlag,fullPrice
2492,12493,None,#N,None,Y,0


`productGroupName` describes a category of products and `productGroupCode` is its short name.

In [31]:
count_rows_by(df_products, by=["productGroupCode","productGroupName"])

,productGroupCode,productGroupName,count
1,AR,ARTWORK,3300
2,BK,BOOK,240
5,GA,GAME,231
0,AP,APPAREL,86
6,OC,OCCASION,71
7,OT,OTHER,55
3,CA,CALENDAR,31
4,FR,FREEBIE,25


## Environment

In [32]:
%load_ext watermark
%watermark -v -n -m -p numpy,scipy,sklearn,pandas,matplotlib,seaborn

Thu Jun 13 2019 

CPython 3.6.4
IPython 7.2.0

numpy 1.15.4
scipy 1.1.0
sklearn 0.20.2
pandas 0.24.0
matplotlib 3.0.2
seaborn 0.9.0

compiler   : GCC 7.2.0
system     : Linux
release    : 4.12.14-lp150.12.61-default
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit
